# Setup

Don't change this cell; setup should remain the same for all data series.

In [1]:
#Imports
import glob, math, time
import numpy as np
import pandas as pd
import lmfit as lf
import statistics as s
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from scipy import signal as sig

#Import custom library included in this repository
from R76Tools import *

#Use local mplstyle
plt.style.use('./default.mplstyle')

# Set up possible data paths
frittspath = "/data/chocula/fritts/data/k100proc/midasrq/byseries/"
neogpath = "/data/chocula/neog/rq/"

## Data Selection
Change this cell in order to select and label data.

In [125]:
ser = ["07220919_1723","07220919_2012","07220920_1007","07220920_2138","07220921_1034","07220921_1631","07220921_2055"]
lab = ["-82V","-82V","-82V","-82V"]
datapath = frittspath #The data is in this folder. change as needed.

# Load and Plot
Don't change these cells unless you are experiencing issues with the selected binning. (In that case, change the first line of the second cell.)

In [ ]:
#Start by importing and organizing our data
e=[]; z=[];
for series in ser:
    loc = frittspath + series + "/umn_*.root"
    e_chain,z_chain = makechain(loc,filters=fittingfilters)
    z_chain = pd.concat([e_chain,z_chain],axis=1)
    z.append(z_chain)
    
for x in z:
    x['crand'] = crand(x)
    x['pt_keV'] = pt_keV(x)
    x['cam'] =  cam(x)
    x['cphi1'] = cphi1(x)
    x['PSUMbs'] = PSUMbs(x)

In [ ]:
bins = [np.linspace(-2300,-1500,200),np.linspace(350,450,150)]
    
#Plot
n = len(ser); vert = math.ceil(np.sqrt(n)); hor = math.ceil(n/math.ceil(np.sqrt(n)))
fig,ax = plt.subplots(vert,hor,figsize=(3*hor,3*vert),sharey=True)
try:
    for i,axarray in enumerate(ax):
        try:
            for j,axis in enumerate(axarray):
                k = j + i*len(axarray)
                if k < len(z): #do nothing if we have an empty corner.
                    axis.hist2d(z[k]['PSUMbs'][~z[k]['crand']]-18000,z[k]['pt_keV'][~z[k]['crand']],bins=bins)
        except TypeError: #Case for when there are multiple entries but only one column.
            k = i
            axarray.hist2d(z[k]['PSUMbs'][~z[k]['crand']]-18000,z[k]['pt_keV'][~z[k]['crand']],bins=bins)
except TypeError: #Case for when there is only one entry.
    if len(z) == 1:
        k = 0 #We could squish z from an array to its entry but that gets complicated later on.
        ax.hist2d(z[k]['PSUMbs'][~z[k]['crand']]-18000,z[k]['pt_keV'][~z[k]['crand']],bins=bins)
    else:
        raise ValueError("axes could not be enumerated twice, or once, but Z had more than one entry... what!!")

fig.supxlabel("Baseline Elevation"); fig.supylabel("Phonon Energy [keV]")
plt.show()

## Define points to fit to
Change the first two lines of this cell until you get the lines looking right in the second.

In [ ]:
points = [[-2200,396],[-2100,385]] #Some points along one of the lines
x = z[0] #Pick one of the entries in z to use
bins2 = [np.linspace(0,3500,200),np.linspace(0,225,150)]

fig = plt.figure()
plt.hist2d(x["PSUMbs"][~x["crand"]]-18000, x["pt_keV"][~x["crand"]]#)
           ,bins=bins)
           #,bins = bins2)
           #bins=[np.linspace(3000,5000,200),np.linspace(0,100,150)])
m = (points[1][1]-points[0][1])/(points[1][0]-points[0][0]); b = points[0][1] - points[0][0]*m
x = np.linspace(points[0][0],points[1][0],1000)
plt.scatter([points[0][0],points[1][0]],[points[0][1],points[1][1]],color='white')
plt.plot(x,m*x+b,color='lime')
plt.xlabel("Baseline Elevation")
plt.ylabel("Phonon Energy [keV]")
plt.show()

## Check baseline correction

Don't change these cells; they are here to show you the results of your correction only.

In [ ]:
for x in z:
    x['bscorr_man'] = bscorr(x,m/b)

In [ ]:
fig = plt.figure()
x = z[0]
plt.hist2d(x["PSUMbs"][~x["crand"]]-18000, x["bscorr_man"][~x["crand"]]#)
           #,bins=bins)
           #bins=bins2)
           ,bins = [np.linspace(-500,10000,300),np.linspace(-5000,15000,300)])
plt.axhline(b,linestyle='--',color='lime')
plt.xlabel("Baseline Elevation")
plt.ylabel("Phonon Energy [keV]")
plt.show()

In [ ]:
#Plot
n = len(ser); vert = math.ceil(np.sqrt(n)); hor = math.ceil(n/math.ceil(np.sqrt(n)))
fig,ax = plt.subplots(vert,hor,figsize=(3*hor,3*vert),sharey=True)
try:
    for i,axarray in enumerate(ax):
        try:
            for j,axis in enumerate(axarray):
                k = j + i*len(axarray)
                if k < len(z): #do nothing if we have an empty corner.
                    axis.hist2d(z[k]['PSUMbs'][~z[k]['crand']]-18000,z[k]['bscorr_man'][~z[k]['crand']],bins=bins)
                    axis.axhline(points[0][1],linestyle='--',color='lime')
                    axis.axhline(points[1][1],linestyle='--',color='lime')
                    axis.axhline(b,linestyle='--',color='lime')
        except TypeError:
            k = i
            axarray.hist2d(z[k]['PSUMbs'][~z[k]['crand']]-18000,z[k]['bscorr_man'][~z[k]['crand']],bins=bins)
            axarray.axhline(points[0][1],linestyle='--',color='lime')
            axarray.axhline(points[1][1],linestyle='--',color='lime')
            axarray.axhline(b,linestyle='--',color='lime')
except TypeError: #Case for when there is only one entry.
    if len(z) == 1:
        k = 0 #We could squish z from an array to its entry but that gets complicated later on.
        ax.hist2d(z[k]['PSUMbs'][~z[k]['crand']]-18000,z[k]['bscorr_man'][~z[k]['crand']],bins=bins)
        ax.axhline(points[0][1],linestyle='--',color='lime')
        ax.axhline(points[1][1],linestyle='--',color='lime')
        ax.axhline(b,linestyle='--',color='lime')
    else:
        raise ValueError("axes could not be enumerated twice or once but Z had more than one array... what!!")
fig.supxlabel("Baseline Elevation"); fig.supylabel("Phonon Energy [keV]")
plt.show()

This cell is for viewing any individual plot; change as desired.

In [ ]:
k = 0
fig = plt.figure()
plt.hist2d(z[k]['PSUMbs'][~z[k]['crand']]-18000,z[k]['bscorr_man'][~z[k]['crand']]#)
           ,bins=bins)
           #bins=bins2)
           #,bins = [np.linspace(-10000,-5000,300),np.linspace(-300,100,300)])
plt.axhline(points[0][1],linestyle='--',color='lime')
plt.axhline(points[1][1],linestyle='--',color='lime')
plt.axhline(b,linestyle='--',color='lime')
plt.axhline(320,linestyle='--',color='lime')
plt.axhline(303,linestyle='--',color='lime')
plt.axhline(295,linestyle='--',color='lime')
plt.axhline(175,linestyle='--',color='lime')
plt.axhline(190,linestyle='--',color='lime')
plt.axhline(150,linestyle='--',color='lime')
plt.axhline(0,linestyle='--',color='lime')
plt.show()

### Save or repeat
Either go back up to the "Define points to fit to" section and adjust your points to fix the correction or save the correction information, as printed below, to `baselinecorrectionvalues.csv`.

In [ ]:
for series in ser:
    print(series+",",str(m)+",",b)

# Experimental

While I wait for things to run I am experimenting with an interactive interface for this.

In [ ]:
#Imports
from matplotlib.widgets import Button
from ipywidgets.widgets import Dropdown, Checkbox
from IPython.display import display, Javascript

#Set up dropdown for series selection
bor = 0
dd = Dropdown(description = "Choose series",layout={'width': 'max-content'},
              options = [(x,i) for i,x in enumerate(ser)],value=bor)
def dropdown_handler(change):
    global bor
    #print(change.new)
    bor = change.new  # This line isn't working
dd.observe(dropdown_handler, names='value')
display(dd)

#Set up plot
fig,ax = plt.subplots()

#Define buttons
b1 = Button(ax,"Point 1",color='orange'); b2 = Button(ax,"Point 2",color='grey')
bdone = Button(ax,"Done!")
def button1(self,event):
    selectingpoint1 = True
    b1.color = 'orange'; b2.color = 'grey'
def button2(self,event):
    selectingpoint1 = False
    b1.color = 'grey'; b2.color = 'orange'
m,b = 0,0; xvals=[0] #placeholder values
def done(self,event):
    global m,b,xvals,z
    m = (points[1][1]-points[0][1])/(points[1][0]-points[0][0])
    b = points[0][1] - points[0][0]*m
    xvals = np.linspace(points[0][0],points[1][0],1000)
    bdone.color = 'b'
    for x in z:
        x['bscorr_man'] = bscorr(x,m/b)
    Javascript("Jupyter.notebook.execute_cells([12])")
b1.on_clicked(button1); b2.on_clicked(button2)
    
#Define click event
point1, point2 = [0,0],[0,0]; autoflip,selectingpoint1 = True, True
def on_pick(event):
    global selectingpoint1, autoflip, point1, point2
    xmouse, ymouse = event.mouseevent.xdata, event.mouseevent.ydata
    if selectingpoint1:
        point1 = [xmouse,ymouse]
        if autoflip:
            b1.color = 'grey'; b2.color = 'orange'
    else:
        point2 = [xmouse,ymouse]
        if autoflip:
            b1.color = 'orange'; b2.color = 'grey'
    if autoflip:
        if not selectingpoint1:
            done()
        selectingpoint1 = !selectingpoint1 #switch to the other point once we've clicked the one we were on
fig.canvas.callbacks.connect('pick_event', on_pick)

#Set up plot
points = [point1,point2]
x = z[bor]
ax.hist2d(x["PSUMbs"][~x["crand"]]-18000, x["pt_keV"][~x["crand"]],
           bins=bins)
           #bins = bins2)
           #bins=[np.linspace(3000,5000,200),np.linspace(0,100,150)])
ax.scatter([points[0][0],points[1][0]],[points[0][1],points[1][1]],color='white')
ax.plot(xvals,m*xvals+b,color='lime')
ax.set_xlabel("Baseline Elevation")
ax.set_ylabel("Phonon Energy [keV]")

#Set up checkboxes
box = Checkbox(True, description="Automatically go to next step.")
display(box)
def change(b):
    global autoflip
    autoflip = !autoflip

#Show plot
fig.show()

In [ ]:
#Set up plot
n = len(ser); vert = math.ceil(np.sqrt(n)); hor = math.ceil(n/math.ceil(np.sqrt(n)))
fig,ax = plt.subplots(vert,hor,figsize=(3*hor,3*vert),sharey=True)
try:
    for i,axarray in enumerate(ax):
        try:
            for j,axis in enumerate(axarray):
                k = j + i*len(axarray)
                if k < len(z): #do nothing if we have an empty corner.
                    axis.hist2d(z[k]['PSUMbs'][~z[k]['crand']]-18000,z[k]['bscorr_man'][~z[k]['crand']],bins=bins)
                    axis.axhline(points[0][1],linestyle='--',color='lime')
                    axis.axhline(points[1][1],linestyle='--',color='lime')
                    axis.axhline(b,linestyle='--',color='lime')
        except TypeError:
            k = i
            axarray.hist2d(z[k]['PSUMbs'][~z[k]['crand']]-18000,z[k]['bscorr_man'][~z[k]['crand']],bins=bins)
            axarray.axhline(points[0][1],linestyle='--',color='lime')
            axarray.axhline(points[1][1],linestyle='--',color='lime')
            axarray.axhline(b,linestyle='--',color='lime')
except TypeError: #Case for when there is only one entry.
    if len(z) == 1:
        k = 0 #We could squish z from an array to its entry but that gets complicated later on.
        ax.hist2d(z[k]['PSUMbs'][~z[k]['crand']]-18000,z[k]['bscorr_man'][~z[k]['crand']],bins=bins)
        ax.axhline(points[0][1],linestyle='--',color='lime')
        ax.axhline(points[1][1],linestyle='--',color='lime')
        ax.axhline(b,linestyle='--',color='lime')
    else:
        raise ValueError("axes could not be enumerated twice or once but Z had more than one array... what!!")
fig.supxlabel("Baseline Elevation"); fig.supylabel("Phonon Energy [keV]")

#Show plot
plt.show()

#Print values
for series in ser:
    print(series+",",str(m)+",",b)